In [2]:
from isaacgym.torch_utils import *
from isaacgym import gymtorch, gymapi, gymutil
import torch
import math

Importing module 'gym_38' (/home/natcha/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /home/natcha/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 2.2.1+cu121
Device count 1
/home/natcha/isaacgym/python/isaacgym/_bindings/src/gymtorch


Using /home/natcha/.cache/torch_extensions/py38_cu121 as PyTorch extensions root...
Emitting ninja build file /home/natcha/.cache/torch_extensions/py38_cu121/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module gymtorch...


# Transformation env frame to 

In [71]:
# base position and orientation express on env frame
x = 0.0
y = 3.0
z = 2.0
roll = torch.tensor([0])
pitch = torch.tensor([math.pi/2])
yaw = torch.tensor([math.pi/2])

orientation = quat_from_euler_xyz(roll, pitch, yaw)
rx = orientation[0][0]
ry = orientation[0][1]
rz = orientation[0][2]
rw = orientation[0][3]

# goal pos on env frame
g_x = 1.0
g_y = 6.0
g_z = 3.0

### 1. Ground truth

In [72]:

transform = gymapi.Transform(p = gymapi.Vec3(x, y, z), r=gymapi.Quat(rx, ry, rz, rw))

transform_inv = transform.inverse()
pos_on_base = transform_inv.transform_point(gymapi.Vec3(g_x, g_y, g_z))
print(pos_on_base)
print(transform.transform_point(pos_on_base))

Vec3(-1.000000, -1.000000, 3.000000)
Vec3(1.000000, 5.999999, 3.000000)


### 2. Apply by my self

In [65]:
base_quat = orientation
root_states = torch.tensor([[x, y, z]])
goal_states = torch.tensor([[g_x, g_y, g_z, 1]])
T = torch.tensor([[1, 0, 0, x], [0, 1, 0, y], [0, 0, 1, z], [0, 0, 0, 1]])
T_inv = torch.inverse(T)

base = (torch.matmul(T_inv, goal_states.T).T)[:, : -1]
base = quat_rotate_inverse(base_quat, base)

print(T_inv)
print(goal_states)
print(root_states)
print(base)

tensor([[ 1.,  0.,  0.,  0.],
        [-0.,  1., -0., -3.],
        [ 0.,  0.,  1., -2.],
        [ 0.,  0.,  0.,  1.]])
tensor([[1., 6., 3., 1.]])
tensor([[0., 3., 2.]])
tensor([[-1.0000,  3.0000,  1.0000]])


### 3. Create function

In [66]:
def transformation_inverse(q:torch.tensor, t:torch.tensor, g:torch.tensor):
    """
    q is orientation of frame2 (agent) express on frame1 (env)
    t is origin position of frame2 (agent) relative to frame1 (env)
    g is point position on frame1 (env)
    """
    t_extend = torch.cat([t, torch.ones(t.shape[0], 1)], dim=-1)
    T_inv = quat_conjugate(t_extend)[:, :-1]
    a = T_inv + g                          # inverse translation
    b = quat_rotate_inverse(q, a)          # inverse rotation
    return b

In [67]:
num_env = 5
root_states = torch.tensor([x, y, z])
goal_pos = torch.tensor([g_x, g_y, g_z])

root_states = root_states.repeat((num_env, 1))
base_quat_real = base_quat.repeat((num_env,1))
goal_pos = goal_pos.repeat((num_env, 1))

transformation_inverse(base_quat_real, root_states, goal_pos)

tensor([[-1.0000,  3.0000,  1.0000],
        [-1.0000,  3.0000,  1.0000],
        [-1.0000,  3.0000,  1.0000],
        [-1.0000,  3.0000,  1.0000],
        [-1.0000,  3.0000,  1.0000]])

In [119]:
base_vel_xy = torch.tensor([[-2.0, -2.0], [2.0, 3.0]])
diff_distance = torch.tensor([[5.0, 5.0], [5.0, 5.0]]) - torch.tensor([[0.0, 0.0], [0.0, 0.0]])

norm_vel = torch.norm(base_vel_xy, dim=1)
norm_distance = torch.norm(diff_distance, dim=1)

dot_product = torch.sum(base_vel_xy * diff_distance, dim=1)

print(dot_product)
print(dot_product / (norm_vel * norm_distance))

tensor([-20.,  25.])
tensor([-1.0000,  0.9806])


In [123]:
test = torch.tensor([[-2.0, -2.0, 1.0], [2.0, 3.0, 2.0]])
print(torch.clip(test, 0, 1))

tensor([[0., 0., 1.],
        [1., 1., 1.]])
